In [ ]:
# Import libraries
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm
import yaml
import matplotlib.pyplot as plt
from ultralytics.engine.results import Results
from ultralytics import YOLO
import numpy as np
from PIL import Image, ExifTags
import torch

In [ ]:
# INPUT_DIRS
INPUT_DATA_DIR = Path('dataset')
## Drop the Folder if it already exists
DATASETS_DIR = Path('dataset')
# Image & labels directory
TRAIN_IMAGES_DIR = DATASETS_DIR / 'images' / 'train'
TRAIN_LABELS_DIR = DATASETS_DIR / 'labels'/ 'train'
TEST_IMAGES_DIR = DATASETS_DIR / 'images' / 'test'
VAL_IMAGES_DIR = DATASETS_DIR / 'images' /'val'
VAL_LABELS_DIR = DATASETS_DIR / 'labels' /'val'

# Load train and test files
train = pd.read_csv(INPUT_DATA_DIR / 'Train_df.csv')
val = pd.read_csv(INPUT_DATA_DIR / 'Val_df.csv')
test = pd.read_csv(INPUT_DATA_DIR / 'Test.csv')
ss = pd.read_csv(INPUT_DATA_DIR / 'SampleSubmission.csv')

class_map = {cls: i for i, cls in enumerate(sorted(train['class'].unique().tolist()))}
# Strip any spacing from the class item and make sure that it is a str
train['class'] = train['class'].str.strip()

# Map {'healthy': 2, 'cssvd': 1, anthracnose: 0}
train['class_id'] = train['class'].map(class_map)

train_df = train
val_df = val

# Create a data.yaml file required by yolo
class_names = sorted(train['class'].unique().tolist())
num_classes = len(class_names)
data_yaml = {
    "path" : str(DATASETS_DIR.absolute()),
    'train': str(TRAIN_IMAGES_DIR.absolute()),
    'val': str(VAL_IMAGES_DIR.absolute()),
    'test': str(TEST_IMAGES_DIR.absolute()),
    'nc': num_classes,
    'names': class_names
}

val_image_names = [str(Path(name).stem) for name in val_df['Image_ID'].unique()]
train_image_names = [str(Path(name).stem) for name in train['ImagePath'].unique()]

In [ ]:
from glob import glob

# Validate the model on the validation set
BEST_PATH = sorted(glob("runs/detect/train*/weights/best.pt"))[-1]
# BEST_PATH = "runs/detect/train3/weights/best.pt"
BEST_PATH

In [ ]:
for flag, v in ExifTags.TAGS.items():
    if v == "Orientation":
        break


def load_image(filepath):
    image = Image.open(filepath)
    # return image

    exif = image._getexif()
    if exif is None:
        return image

    orientation_value = exif.get(flag, None)

    if orientation_value == 3:
        image = image.rotate(180, expand=True)
    elif orientation_value == 6:
        image = image.rotate(270, expand=True)
    elif orientation_value == 8:
        image = image.rotate(90, expand=True)
    return image


flag

In [ ]:
# Validate the model on the validation set
BEST_CFG = sorted(glob("runs/detect/train*/args.yaml"))[-1]
BEST_CFG

In [ ]:
# Load the trained YOLO model
model = YOLO(BEST_PATH)

# Path to the test images directory
test_dir_path = TEST_IMAGES_DIR

# Get a list of all image files in the test directory
image_files = os.listdir(test_dir_path)

# Initialize an empty list to store the results for all images
all_data = []

# Initialize an empty list to store the results for all images
all_data = []

# Batch size for predictions
batch_size = 16

# Process images in batches
for i in tqdm(range(0, len(image_files), batch_size)):
	batch_files = image_files[i:i + batch_size]
	batch_images = [load_image(os.path.join(test_dir_path, img_file)) for img_file in batch_files]

	# Make predictions on the batch of images
	results = model.predict(
		batch_images,
		conf=0.,
		imgsz=1024,
		batch=batch_size,
		verbose=False,
		device="cuda:0",

		seed=0,
		deterministic=True,
		close_mosaic=10,
		mask_ratio=4,
		iou=.7,
		max_det=100,
		format="torchscript",
		nms=True,
		cfg=BEST_CFG,
	)

	# Iterate through each result in the batch
	for img_file, result in zip(batch_files, results):
		boxes = result.boxes.xyxy.tolist() if result.boxes else []  # Bounding boxes in xyxy format
		classes = result.boxes.cls.tolist() if result.boxes else []  # Class indices
		confidences = result.boxes.conf.tolist() if result.boxes else []  # Confidence scores
		names = result.names  # Class names dictionary

		if boxes:  # If detections are found
			for box, cls, conf in zip(boxes, classes, confidences):
				x1, y1, x2, y2 = box
				detected_class = names[int(cls)]  # Get the class name from the names dictionary

				# Add the result to the all_data list
				all_data.append(
					{
						"Image_ID": str(img_file),
						"class": detected_class,
						"confidence": conf,
						"ymin": y1,
						"xmin": x1,
						"ymax": y2,
						"xmax": x2,
					}
				)
		else:  # If no objects are detected
			all_data.append(
				{
					"Image_ID": str(img_file),
					"class": "None",
					"confidence": None,
					"ymin": None,
					"xmin": None,
					"ymax": None,
					"xmax": None,
				}
			)


In [ ]:
# Convert the list to a DataFrame for all images
sub = pd.DataFrame(all_data)

In [ ]:
sub.head()

In [ ]:
sub.describe()

In [ ]:
sub['class'].value_counts()

In [ ]:
sub.isna().sum()

class
healthy        1153
cssvd           801
anthracnose     694
None             57
Name: count, dtype: int6

In [ ]:
sub.to_csv("dataset/predictions/05-predictions.csv", index=False)

In [ ]:
sub["confidence"].describe()

In [ ]:
import pandas as pd

sub = pd.read_csv('dataset/predictions/05-predictions.csv')

sub.sample(6)

In [ ]:
sub["Image_ID"].value_counts().describe()

In [ ]:
sub["Image_ID"].nunique()

In [ ]:
sub.isna().sum()